# ShoppingMall Distance

In [29]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [30]:
shopping_mall = pd.read_csv('../../ADS ASS2/Data/shopping_malls.csv')
apartment_data = pd.read_csv('../../ADS ASS2/Data/updated_domain_with_lat_long.csv')

In [31]:
print(len(shopping_mall))
print(shopping_mall.columns)
print(shopping_mall.head)

641
Index(['Name', 'Address', 'Longitude', 'Latitude'], dtype='object')
<bound method NDFrame.head of                            Name                                       Address  \
0           Westfield Doncaster                 619 Doncaster Road, Doncaster   
1           Greensborough Plaza                 25 Main Street, Greensborough   
2       Westfield Fountain Gate            25/55 Overland Drive, Narre Warren   
3        South Melbourne Market      322-326 Coventry Street, South Melbourne   
4                 DFO Moorabbin  250 Centre Dandenong Road, Moorabbin Airport   
..                          ...                                           ...   
636             Backhaus Arcade                314 Lyttleton Terrace, Bendigo   
637               Abbott Arcade                       7 Queen Street, Bendigo   
638               Epsom Village                       22 Howard Street, Epsom   
639  Bendigo Showgrounds Market                    Holmes Road, North Bendigo   
640    

In [32]:
shopping_mall.rename(columns={
    'Longitude': 'Mall_Longitude',
    'Latitude': 'Mall_Latitude'
}, inplace=True)

print(len(shopping_mall))
print(shopping_mall)

641
                           Name                                       Address  \
0           Westfield Doncaster                 619 Doncaster Road, Doncaster   
1           Greensborough Plaza                 25 Main Street, Greensborough   
2       Westfield Fountain Gate            25/55 Overland Drive, Narre Warren   
3        South Melbourne Market      322-326 Coventry Street, South Melbourne   
4                 DFO Moorabbin  250 Centre Dandenong Road, Moorabbin Airport   
..                          ...                                           ...   
636             Backhaus Arcade                314 Lyttleton Terrace, Bendigo   
637               Abbott Arcade                       7 Queen Street, Bendigo   
638               Epsom Village                       22 Howard Street, Epsom   
639  Bendigo Showgrounds Market                    Holmes Road, North Bendigo   
640                 COLBINABBIN                          34 C345, Colbinabbin   

     Mall_Longitude  Ma

In [33]:
print(len(apartment_data))

13453


## Calculate the ShoppingMall Distance

In [36]:
apartment_data['geometry'] = apartment_data.apply(lambda x: Point((x['Longitude'], x['Latitude'])), axis=1)
apartment_gdf = gpd.GeoDataFrame(apartment_data, geometry='geometry', crs="EPSG:4326") 
apartment_gdf_valid = apartment_gdf[~apartment_gdf.is_empty]


shopping_mall['geometry'] = shopping_mall.apply(lambda x: Point((x['Mall_Longitude'], x['Mall_Latitude'])), axis=1)
mall_gdf = gpd.GeoDataFrame(shopping_mall, geometry='geometry', crs="EPSG:4326")

apartment_gdf_valid_3857 = apartment_gdf_valid.to_crs(epsg=3857)
mall_gdf_3857 = mall_gdf.to_crs(epsg=3857)

apartment_gdf_with_mall = gpd.sjoin_nearest(
    apartment_gdf_valid_3857,
    mall_gdf_3857,
    how='left'
)

apartment_gdf_with_mall.reset_index(inplace=True)

apartment_gdf_with_mall = apartment_gdf_with_mall.sort_values(by='index').drop_duplicates(subset='index', keep='first')

print(len(apartment_gdf_with_mall))
print(apartment_gdf_with_mall.head())


13453
   index          Price                                   Address_left  Beds  \
0      0  $270 per week     50 Elmhurst Road, Bayswater North VIC 3153     1   
5      1            280        63 Pintail Drive, Melton South VIC 3338     1   
6      2            300         8 Jacobena Road, Melton South VIC 3338     1   
7      3        $400.00       126 Portarlington Road, Newcomb VIC 3219     3   
9      4        $395.00  123 Mossfiel Drive, Hoppers Crossing VIC 3029     3   

   Baths  Parking   Type  Weekly Price   Latitude   Longitude  \
0      1        0  House           270 -37.835649  145.264776   
5      1        0  House           280 -37.719884  144.580003   
6      1        0  House           300 -37.714715  144.565848   
7      1        1  House           400 -38.161122  144.399746   
9      2        2  House           395 -37.867435  144.690112   

                            geometry  index_right                      Name  \
0  POINT (16170800.850 -4556234.531)       

In [41]:
import googlemaps
def calculate_driving_distance(row):
    gmaps = googlemaps.Client(key='YOUR_API_KEY')
    apartment_coords = f"{row['Latitude']},{row['Longitude']}" 
    mall_coords = f"{row['Mall_Latitude']},{row['Mall_Longitude']}" 
    
    try:
        result = gmaps.distance_matrix(apartment_coords, mall_coords, mode='driving')
        
        if 'distance' in result['rows'][0]['elements'][0]:
            distance_meters = result['rows'][0]['elements'][0]['distance']['value']
            distance_km = distance_meters / 1000
            return distance_km
        else:
            print(f"No distance data available for {row['Address_left']} to {row['Address_right']}")
            return None
        
    except Exception as e:
        print(f"Error calculating distance from {row['Address_left']} to {row['Address_right']}: {e}")
        return None

# Apply the function to each row in the DataFrame
apartment_gdf_with_mall['distance_to_train'] = apartment_gdf_with_mall.apply(calculate_driving_distance, axis=1)

# Display the updated DataFrame
print(apartment_gdf_with_mall.head())


No distance data available for 5, 6 & 11/18 Gerald Street, Murrumbeena VIC 3163 to 120 Wilson Street, Horsham
   index          Price                                   Address_left  Beds  \
0      0  $270 per week     50 Elmhurst Road, Bayswater North VIC 3153     1   
5      1            280        63 Pintail Drive, Melton South VIC 3338     1   
6      2            300         8 Jacobena Road, Melton South VIC 3338     1   
7      3        $400.00       126 Portarlington Road, Newcomb VIC 3219     3   
9      4        $395.00  123 Mossfiel Drive, Hoppers Crossing VIC 3029     3   

   Baths  Parking   Type  Weekly Price   Latitude   Longitude  \
0      1        0  House           270 -37.835649  145.264776   
5      1        0  House           280 -37.719884  144.580003   
6      1        0  House           300 -37.714715  144.565848   
7      1        1  House           400 -38.161122  144.399746   
9      2        2  House           395 -37.867435  144.690112   

                  

In [43]:
apartment_gdf_with_mall.rename(columns={'distance_to_train': 'distance_to_mall'}, inplace=True)
apartment_gdf_with_mall.to_csv('../../ADS ASS2/Data/apartment_with_mall.csv', index=False)

## Calculate the Correlations of ShoppingMall

In [44]:
apartment_data_mall = apartment_gdf_with_mall[apartment_gdf_with_mall['Type'] == 'Apartment']

correlation_apartment = apartment_data_mall[['Weekly Price', 'distance_to_mall']].corr()

print("Correlation between Weekly Price and Distance to Mall:")
print(correlation_apartment)

Correlation between Weekly Price and Distance to Mall:
                  Weekly Price  distance_to_mall
Weekly Price          1.000000         -0.147185
distance_to_mall     -0.147185          1.000000


In [46]:
house_data_mall = apartment_gdf_with_mall[apartment_gdf_with_mall['Type'] != 'Apartment']

correlation_house = house_data_mall[['Weekly Price', 'distance_to_mall']].corr()

print("Correlation between Weekly Price and Distance to Mall:")
print(correlation_house)

Correlation between Weekly Price and Distance to Mall:
                  Weekly Price  distance_to_mall
Weekly Price          1.000000         -0.131561
distance_to_mall     -0.131561          1.000000
